In [1]:
!pip install resume-parser

You should consider upgrading via the 'c:\python38\python.exe -m pip install --upgrade pip' command.


In [2]:
!pip install https://github.com/explosion/spacy-models/releases/download/en_core_web_sm-2.3.1/en_core_web_sm-2.3.1.tar.gz
!pip install importlib-metadata==3.2.0

You should consider upgrading via the 'c:\python38\python.exe -m pip install --upgrade pip' command.
You should consider upgrading via the 'c:\python38\python.exe -m pip install --upgrade pip' command.


In [ ]:
!pip install pyodbc

In [3]:
import nltk

nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('universal_tagset')
nltk.download('maxent_ne_chunker')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('brown')

[nltk_data] Downloading package punkt to C:\Users\Ashlin Darius
[nltk_data]     G\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\Ashlin Darius G\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package universal_tagset to C:\Users\Ashlin
[nltk_data]     Darius G\AppData\Roaming\nltk_data...
[nltk_data]   Package universal_tagset is already up-to-date!
[nltk_data] Downloading package maxent_ne_chunker to C:\Users\Ashlin
[nltk_data]     Darius G\AppData\Roaming\nltk_data...
[nltk_data]   Package maxent_ne_chunker is already up-to-date!
[nltk_data] Downloading package stopwords to C:\Users\Ashlin Darius
[nltk_data]     G\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to C:\Users\Ashlin Darius
[nltk_dat

True

In [4]:
import spacy
spacy.__version__

'2.3.5'

In [5]:
import en_core_web_sm
en_core_web_sm.__version__

'2.3.1'

In [6]:
import pyodbc

def RunQuery(query):
 try: 
  conn = pyodbc.connect('Driver={SQL Server};'
                      'Server=localhost;'
                      'Database=CVs;'
                      'Trusted_Connection=yes;')
  cursor = conn.cursor()

  cursor.execute(query)
  cursor.commit()
  return "Query Executed"
 except Exception as e:
  return "Database Error",e


In [7]:
def fields(cur):
    results = {}
    column = 0
    for d in cur.description:
        results[d[0]] = column
        column = column + 1

    return results




def CheckIfDataExists(name):
  bValid = False 
  conn = pyodbc.connect('Driver={SQL Server};'
                      'Server=localhost;'
                      'Database=CVs;'
                      'Trusted_Connection=yes;')
  cursor = conn.cursor()

  cursor.execute("select * from tblUsers")  
  field_map = fields(cursor) 
  for row in cursor:
   if row[field_map['name']] == name:   
    bValid = True
  
  return bValid
  

def getID(name):
  conn = pyodbc.connect('Driver={SQL Server};'
                      'Server=localhost;'
                      'Database=CVs;'
                      'Trusted_Connection=yes;')
  cursor = conn.cursor()

  cursor.execute("select * from tblUsers")  
  field_map = fields(cursor) 
  for row in cursor:
   if row[field_map['name']] == name:   
    id = row[field_map['personid']]
  
  return id


 



In [8]:
def ProcessCV():
 for index,row in df.iterrows():
  email = "'"+row['email']+"'"
  phone = "'"+row['phone']+"'"
  name  = "'"+row['name']+"'"
  total_exp =  "'"+str(row['total_exp'])+"'"
  university = row['university']
  designition = row['designition']
  degree = row['degree']
  skills =  row['skills']
  companies = row['Companies worked at']

  bValid = CheckIfDataExists(row['name'])
 


  if bValid == False:
    try: 
     conn = pyodbc.connect('Driver={SQL Server};'
                       'Server=localhost;'
                       'Database=CVs;'
                       'Trusted_Connection=yes;')
     cursor = conn.cursor()
     sql =  f'''
     INSERT INTO tblUsers 
     (name,phone,total_exp,email,password,address) 
     VALUES ({name},{phone},{total_exp},{email},'','')
     
     '''
     cursor.execute(sql)
    
     conn.commit()
    
     id = getID(row['name'])
     personid = "'"+str(id)+"'"
     for x in university:
      universites = "'"+x+"'"
      sql = f''' 
      INSERT INTO tblUniversity
      (personid,university) 
      VALUES ({personid},{university})
      '''    
      RunQuery(sql)

     for x in skills:
      skill = "'"+x.lstrip()+"'"
      sql = f''' 
      INSERT INTO tblSkills 
      (personid,Skills) 
      VALUES ({personid},{skill})
      '''    
      RunQuery(sql)
 
     for x in designition:
      design = "'"+x+"'"
      sql = f''' 
      INSERT INTO tblDesignition
      (personid,designition) 
      VALUES ({personid},{design})
      '''    
      RunQuery(sql)

     for x in degree:
      deg = "'"+x+"'"
      sql = f''' 
      INSERT INTO tblDegree
      (personid,degree,StartDate,EndDate,Institute) 
      VALUES ({personid},{deg},'','','')
      '''    
      RunQuery(sql)
 
     for x in companies:
      comp = "'"+x+"'"
      sql = f''' 
      INSERT INTO tblCompanies 
      (personid,CompanyName,Position,StartDate,EndDate) 
      VALUES ({personid},{comp},'','','')
      '''    
      RunQuery(sql)

     print("Record inserted")
    except Exception as e:
     print('Exception ',e)
  else:
     print("No new records inserted")



In [9]:
import posixpath
import os
from resume_parser import resumeparse
import pandas as pd


file1 = open('cv.cvs', 'r') 
filedata = file1.readlines() 
paths = []
for i in range(len(filedata)):
 if filedata[i] == "":
  pass  
 else:
   paths.append(filedata[i].replace('\n',''))
 
for x in range(len(paths)):
 paths[x] = paths[x].replace(os.sep,posixpath.sep)
 data = resumeparse.read_file(paths[x])
 data_items = data.items()
 data_list = list(data)
 df = pd.DataFrame([data])
 ProcessCV()
print("Program Completed")
  

C:\Users\Ashlin Darius G\anaconda3\lib\site-packages\spacy\util.py:275: UserWarning: [W031] Model 'en_training' (0.0.0) requires spaCy v2.2 and is incompatible with the current spaCy version (2.3.5). This may lead to unexpected results or runtime errors. To resolve this, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)
2021-05-24 15:14:25,555 [MainThread  ] [WARNI]  Failed to see startup log message; retrying...
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Program Completed
